# Santos Dumont (SD) - F2PY CPU sequencial B710

In [1]:
%%writefile f2py_seq.f90
subroutine st(n, energy, niters, heat, t)
    integer, intent(in) :: n, energy, niters
    double precision, intent(out) :: heat, t   
    integer, parameter :: nsources=3
    integer :: iters, i, j, x, y, sizeArray, sizeStart, sizeEnd
    integer, dimension(3, 2) :: sources
    double precision, allocatable :: aold(:,:), anew(:,:)
    double precision :: t1=0.0, t2=0.0

    call cpu_time(t1)

    sizeArray = n + 2
    sizeStart = 2
    sizeEnd = n + 1

    allocate(aold(sizeArray, sizeArray))
    allocate(anew(sizeArray, sizeArray))
    aold = 0.0
    anew = 0.0
    
    sources(1,:) = (/ n/2,   n/2   /)
    sources(2,:) = (/ n/3,   n/3   /)
    sources(3,:) = (/ n*4/5, n*8/9 /)
    
    do iters = 1, niters, 2
        do j = sizeStart, sizeEnd
            do i = sizeStart, sizeEnd
                anew(i,j) = aold(i,j)/2.0 + (aold(i-1,j) + aold(i+1,j) +  &
                            aold(i,j-1) + aold(i,j+1)) / 4.0 / 2.0
            enddo
        enddo
        do i = 1, nsources
            x = sources(i,1) + 1
            y = sources(i,2) + 1
            anew(x,y) =  anew(x,y) + energy
        enddo
        do j = sizeStart, sizeEnd
            do i = sizeStart, sizeEnd
                aold(i,j) = anew(i,j)/2.0 + (anew(i-1,j) + anew(i+1,j) +  &
                            anew(i,j-1) + anew(i,j+1)) / 4.0 / 2.0
            enddo
        enddo
        do i = 1, nsources
            x = sources(i,1) + 1
            y = sources(i,2) + 1
            aold(x,y) = aold(x,y) + energy
        enddo
    enddo
    heat = 0.0
    do j = sizeStart, sizeEnd
        do i = sizeStart, sizeEnd
            heat = heat + aold(i,j)
        end do
    end do
    deallocate(aold)
    deallocate(anew)
    call cpu_time(t2)
    t = t2 - t1
end subroutine

Overwriting f2py_seq.f90


## Compila e transforma em biblioteca

In [2]:
%%bash
f2py  -c f2py_seq.f90  -m f2py_seq  --opt='-O3'  --quiet  \
      -DNPY_NO_DEPRECATED_API=NPY_1_7_API_VERSION

## Testa a biblioteca criada para ver se funciona

In [3]:
from f2py_seq import st
print(st.__doc__)

heat,t = st(n,energy,niters)

Wrapper for ``st``.

Parameters
----------
n : input int
energy : input int
niters : input int

Returns
-------
heat : float
t : float



## Programa Python que chama a biblioteca

In [4]:
%%writefile f2py_seq.py
from time import time
from f2py_seq import st

n       = 4800    # nxn grid; 4800,1,500→1500; 100,1,10→30; [4800]
energy  = 1       # energy to be injected per iteration; [1]
niters  = 500     # number of iterations; [500]
t0      = -time()

heat, t = st(n, energy, niters)

t0 += time()
print("Heat = %0.4f | Time = %0.4f | TimePyt = %0.4f" %(heat, t, t0))

Overwriting f2py_seq.py


## Roda em nó de execução

Copia o executável para /scratch

In [12]:
! cp f2py_seq* /scratch${PWD#/prj}

cp: aviso: o arquivo de origem “f2py_seq.py” foi especificado mais de uma vez


Arquivo de lote de submissão

In [5]:
%%writefile f2py_seq.srm
#!/bin/bash
#SBATCH --job-name f2py_seq    # Job name
#SBATCH --partition cpu_dev    # Select partition
#SBATCH --ntasks=1             # Total tasks
#SBATCH --time=00:05:00        # Limit execution time
#SBATCH --exclusive            # Exclusive acccess to nodes

echo '========================================'
echo '- Job ID:' $SLURM_JOB_ID
echo '- Tasks per node:' $SLURM_NTASKS_PER_NODE
echo '- # of nodes in the job:' $SLURM_JOB_NUM_NODES
echo '- # of tasks:' $SLURM_NTASKS
echo '- Dir from which sbatch was invoked:' ${SLURM_SUBMIT_DIR##*/}
echo -n '- List of nodes allocated to the job: '
nodeset -e $SLURM_JOB_NODELIST

# Environment                                              
cd
BASE=/scratch${PWD#/prj}
cd $BASE/tama

# Modules
echo '-- modules ----------------------------'
module load anaconda3
source $(conda info --base)/etc/profile.d/conda.sh
export PYTHONNOUSERSITE=1  # include pip user packages
conda activate base
conda activate $BASE/env2  # clone of anaconda3, plus mpi4py

# Executable
EXEC="python f2py_seq.py"

# Start
echo '-- run --------------------------------'
echo '$ srun -n' $SLURM_NTASKS ${EXEC##*/}
echo '-- output -----------------------------'
srun -n $SLURM_NTASKS $EXEC
echo '~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~'

Overwriting f2py_seq.srm


## Envia para a fila de execução

### 1 processo (sequencial)

In [6]:
! sbatch  --ntasks=1  f2py_seq.srm

Submitted batch job 10648031


In [7]:
! squeue  --name f2py_seq --format "%.19S  %.7i  %.10P %.5j  %.2t %.5M %.5D %.4C"

         START_TIME    JOBID   PARTITION  NAME  ST  TIME NODES CPUS
2022-09-22T16:18:50  1064803     cpu_dev f2py_   R  0:01     1   24


In [12]:
! squeue  --name f2py_seq  --format "%.19S  %.7i  %.10P %.5j  %.2t %.5M %.5D %.4C"

         START_TIME    JOBID   PARTITION  NAME  ST  TIME NODES CPUS


In [13]:
! head -20 /scratch${PWD#/prj}/slurm-10648031.out

- Job ID: 10648031
- Tasks per node:
- # of nodes in the job: 1
- # of tasks: 1
- Dir from which sbatch was invoked: tama
- List of nodes allocated to the job: sdumont1309
-- modules ----------------------------
-- run --------------------------------
$ srun -n 1 python f2py_seq.py
-- output -----------------------------
Heat = 1500.0000 | Time = 17.3343 | TimePyt = 17.3353
~~ end ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
